In [1]:
import sys
sys.path.append('..')



In [22]:
import os
import torch
import numpy as np
import pandas as pd
import easydict
import argparse
import json
from pororo import Pororo
from itertools import permutations
from transformers import BertTokenizer
from transformers import logging
import requests
from korre.model import KREModel


class KorRE:
    def __init__(self):
        self.args = easydict.EasyDict({'bert_model': 'datawhales/korean-relation-extraction', 'mode': 'ALLCC', 
                                        'n_class': 97, 'max_token_len': 512, 'max_acc_threshold': 0.6})
        self.ner_module = Pororo(task='ner', lang='ko')
        
        logging.set_verbosity_error()

        self.tokenizer = BertTokenizer.from_pretrained(self.args.bert_model)
        
        # # entity markers tokens
        # special_tokens_dict = {'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}
        # num_added_toks = self.tokenizer.add_special_tokens(special_tokens_dict)   # num_added_toks: 4
        
        self.trained_model = self.__get_model()
        
        # relation id to label
        r = requests.get('https://raw.githubusercontent.com/datawhales/Korean_RE/main/data/relation/relid2label.json')
        self.relid2label = json.loads(r.text)
        
        # relation list
        self.relation_list = list(self.relid2label.keys())

        # device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.trained_model = self.trained_model.to(self.device)
        
    def __get_model(self):
        """ 사전학습된 한국어 관계 추출 모델을 로드하는 함수.
        """
        # trained_model = KREModel.load_from_checkpoint('./ckpt/best-checkpoint.ckpt', args=self.args)
        trained_model = torch.load('../final/entire_model.pth')
        trained_model.eval()
#         trained_model.freeze()

        return trained_model
    
    def __idx2relid(self, idx_list):
        """ onehot label에서 1인 위치 인덱스 리스트를 relation id 리스트로 변환하는 함수.
        
        Example:
            relation_list = ['P17', 'P131', 'P530', ...] 일 때,
            __idx2relid([0, 2]) => ['P17', 'P530'] 을 반환.
        """
        label_out = []

        for idx in idx_list:
            label = self.relation_list[idx]
            label_out.append(label)
            
        return label_out

    def pororo_ner(self, sentence: str):
        """ pororo의 ner 모듈을 이용하여 그대로 반환하는 함수.
        """
        return self.ner_module(sentence)
        
    def ner(self, sentence: str):
        """ 주어진 문장에서 pororo의 ner 모듈을 이용해 개체명 인식을 수행하고 각 개체의 인덱스 위치를 함께 반환하는 함수.
        """
        ner_result = self.ner_module(sentence)
        ner_result = [(item[0], item[1], len(item[0])) for item in ner_result]
        
        modified_list = []
        tmp_cnt = 0

        for item in ner_result:
            modified_list.append((item[0], item[1], [tmp_cnt, tmp_cnt + item[2]]))
            tmp_cnt += item[2]
        
        ent_list = [item for item in modified_list if item[1] != 'O']
        
        return ent_list
    
    def get_all_entity_pairs(self, sentence: str) -> list:
        """ 주어진 문장에서 개체명 인식을 통해 모든 가능한 [문장, subj_range, obj_range]의 리스트를 반환하는 함수.
        
        Example:
            sentence = '모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.'
            
        Return: 
            [(('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
             (('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('안드로이드', 'TERM', [32, 37])),
             (('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('스마트폰', 'TERM', [38, 42])),
             (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
             (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('안드로이드', 'TERM', [32, 37])),
             (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('스마트폰', 'TERM', [38, 42])),
             (('안드로이드', 'TERM', [32, 37]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
             (('안드로이드', 'TERM', [32, 37]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
             (('안드로이드', 'TERM', [32, 37]), ('스마트폰', 'TERM', [38, 42])),
             (('스마트폰', 'TERM', [38, 42]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
             (('스마트폰', 'TERM', [38, 42]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
             (('스마트폰', 'TERM', [38, 42]), ('안드로이드', 'TERM', [32, 37]))]
        """
        # 너무 긴 문장의 경우 500자 이내로 자름
        if len(sentence) >= 500:
            sentence = sentence[:499]
        
        ner_result = self.ner_module(sentence)
        
        # 인식된 각 개체명의 range 계산
        ner_result = [(item[0], item[1], len(item[0])) for item in ner_result]
        
        modified_list = []
        tmp_cnt = 0

        for item in ner_result:
            modified_list.append((item[0], item[1], [tmp_cnt, tmp_cnt + item[2]]))
            tmp_cnt += item[2]
            
        # NER
        ent_list = [item for item in modified_list if item[1] != 'O']
        
        result_list = []

        pairs = list(permutations(ent_list, 2))
        
        return pairs

    def get_all_inputs(self, sentence: str) -> list:
        """ 주어진 문장에서 관계 추출 모델에 통과시킬 수 있는 모든 input의 리스트를 반환하는 함수.
        
        Example:
            sentence = '모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.'
            
        Return:
            [['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [12, 21]],
            ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [32, 37]],
            ..., ]
        """
        pairs = self.get_all_entity_pairs(sentence)
        return [[sentence, ent_subj[2], ent_obj[2]] for ent_subj, ent_obj in pairs]

    def entity_markers_added(self, sentence: str, subj_range: list, obj_range: list) -> str:
        """ 문장과 관계를 구하고자 하는 두 개체의 인덱스 범위가 주어졌을 때 entity marker token을 추가하여 반환하는 함수.
        
        Example:
            sentence = '모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.'
            subj_range = [0, 10]   # sentence[subj_range[0]: subj_range[1]] => '모토로라 레이저 M'
            obj_range = [12, 21]   # sentence[obj_range[0]: obj_range[1]] => '모토로라 모빌리티'
            
        Return:
            '[E1] 모토로라 레이저 M [/E1] 는  [E2] 모토로라 모빌리티 [/E2] 에서 제조/판매하는 안드로이드 스마트폰이다.'
        """
        result_sent = ''
        
        for i, char in enumerate(sentence):
            if i == subj_range[0]:
                result_sent += ' [E1] '
            elif i == subj_range[1]:
                result_sent += ' [/E1] '
            if i == obj_range[0]:
                result_sent += ' [E2] '
            elif i == obj_range[1]:
                result_sent += ' [/E2] '
            result_sent += sentence[i]
        if subj_range[1] == len(sentence):
            result_sent += ' [/E1]'
        elif obj_range[1] == len(sentence):
            result_sent += ' [/E2]'
        
        return result_sent.strip()

    def infer(self, sentence: str, subj_range=None, obj_range=None, entity_markers_included=False):
        """ 입력받은 문장에 대해 관계 추출 태스크를 수행하는 함수.
        """
        # entity marker token이 포함된 경우
        if entity_markers_included:
            # subj, obj name 구하기
            tmp_input_ids = self.tokenizer(sentence)['input_ids']

            if tmp_input_ids.count(20000) != 1 or tmp_input_ids.count(20001) != 1 or \
            tmp_input_ids.count(20002) != 1 or tmp_input_ids.count(20003) != 1:
                raise Exception("Incorrect number of entity marker tokens('[E1]', '[/E1]', '[E2]', '[/E2]').")

            subj_start_id, subj_end_id = tmp_input_ids.index(20000), tmp_input_ids.index(20001)
            obj_start_id, obj_end_id = tmp_input_ids.index(20002), tmp_input_ids.index(20003)

            subj_name = self.tokenizer.decode(tmp_input_ids[subj_start_id+1:subj_end_id])
            obj_name = self.tokenizer.decode(tmp_input_ids[obj_start_id+1:obj_end_id])

            encoding = self.tokenizer.encode_plus(
                             sentence,
                             add_special_tokens=True,
                             max_length=self.args.max_token_len,
                             return_token_type_ids=False,
                             padding='max_length',
                             truncation=True,
                             return_attention_mask=True,
                             return_tensors="pt")

            input_ids = encoding['input_ids'].to(self.device)
            mask = encoding['attention_mask'].to(self.device)

            _, prediction = self.trained_model(input_ids, mask)

            predictions = [prediction.flatten()]
            predictions = torch.stack(predictions).detach().cpu()

            y_pred = predictions.numpy()
            upper, lower = 1, 0
            y_pred = np.where(y_pred > self.args.max_acc_threshold, upper, lower)

            preds_list = []

            for i in range(len(y_pred)):
                class_pred = self.__idx2relid(np.where(y_pred[i]==1)[0])
                preds_list.append(class_pred)

            preds_list = preds_list[0]

            pred_rel_list = [self.relid2label[pred] for pred in preds_list]               

            return [(subj_name, obj_name, pred_rel) for pred_rel in pred_rel_list]

        # entity_markers_included=False인 경우
        else:
            # entity marker가 문장에 포함된 경우
            tmp_input_ids = self.tokenizer(sentence)['input_ids']
            
            if tmp_input_ids.count(20000) >= 1 or tmp_input_ids.count(20001) >= 1 or \
            tmp_input_ids.count(20002) >= 1 or tmp_input_ids.count(20003) >= 1:
                raise Exception("Entity marker tokens already exist in the input sentence. Try 'entity_markers_included=True'.")
            
            # subj range와 obj range가 주어진 경우
            if subj_range is not None and obj_range is not None:
                # add entity markers
                converted_sent = self.entity_markers_added(sentence, subj_range, obj_range)

                encoding = self.tokenizer.encode_plus(
                             converted_sent,
                             add_special_tokens=True,
                             max_length=self.args.max_token_len,
                             return_token_type_ids=False,
                             padding='max_length',
                             truncation=True,
                             return_attention_mask=True,
                             return_tensors="pt")
                
                input_ids = encoding['input_ids'].to(self.device)
                mask = encoding['attention_mask'].to(self.device)
                
                _, prediction = self.trained_model(input_ids, mask)

                predictions = [prediction.flatten()]
                predictions = torch.stack(predictions).detach().cpu()

                y_pred = predictions.numpy()
                upper, lower = 1, 0
                y_pred = np.where(y_pred > self.args.max_acc_threshold, upper, lower)

                preds_list = []

                for i in range(len(y_pred)):
                    class_pred = self.__idx2relid(np.where(y_pred[i]==1)[0])
                    preds_list.append(class_pred)

                preds_list = preds_list[0]

                pred_rel_list = [self.relid2label[pred] for pred in preds_list]

                return [(sentence[subj_range[0]:subj_range[1]], sentence[obj_range[0]:obj_range[1]], pred_rel) for pred_rel in pred_rel_list]

            # 문장만 주어진 경우: 모든 경우에 대해 inference 수행
            else:
                input_list = self.get_all_inputs(sentence)

                converted_sent_list = [self.entity_markers_added(*input_list[i]) for i in range(len(input_list))]

                encoding_list = []

                for i, converted_sent in enumerate(converted_sent_list):
                    tmp_encoding = self.tokenizer.encode_plus(
                                            converted_sent,
                                            add_special_tokens=True,
                                             max_length=self.args.max_token_len,
                                             return_token_type_ids=False,
                                             padding='max_length',
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_tensors="pt"
                                        )
                    encoding_list.append(tmp_encoding)

                predictions = []

                for i, item in enumerate(encoding_list):
                    _, prediction = self.trained_model(
                        item['input_ids'].to(self.device),
                        item['attention_mask'].to(self.device)
                    )

                    predictions.append(prediction.flatten())

                if predictions:
                    predictions = torch.stack(predictions).detach().cpu()

                    y_pred = predictions.numpy()
                    upper, lower = 1, 0
                    y_pred = np.where(y_pred > self.args.max_acc_threshold, upper, lower)

                    preds_list = []
                    for i in range(len(y_pred)):
                        class_pred = self.__idx2relid(np.where(y_pred[i]==1)[0])
                        preds_list.append(class_pred)

                    result_list = []
                    for i, input_i in enumerate(input_list):
                        tmp_subj_range, tmp_obj_range = input_i[1], input_i[2]
                        result_list.append((sentence[tmp_subj_range[0]:tmp_subj_range[1]], sentence[tmp_obj_range[0]:tmp_obj_range[1]], preds_list[i]))

                    final_list = []
                    for tmp_subj, tmp_obj, tmp_list in result_list:
                        for i in range(len(tmp_list)):
                            final_list.append((tmp_subj, tmp_obj, tmp_list[i]))

                    return [(item[0], item[1], self.relid2label[item[2]]) for item in final_list]

                else: return []

            
            

In [23]:
korre = KorRE()

In [24]:
sent1 = '아이폰은 애플에서 만들어진 스마트폰이다.'
sent2 = '모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.'
sent3 = '징크스는 리그오브레전드 캐릭터이다.'

In [291]:
korre.pororo_ner(sent1)

[('아이폰', 'ARTIFACT'),
 ('은', 'O'),
 (' ', 'O'),
 ('애플', 'ORGANIZATION'),
 ('에서', 'O'),
 (' ', 'O'),
 ('만들어진', 'O'),
 (' ', 'O'),
 ('스마트폰', 'TERM'),
 ('이다.', 'O')]

In [210]:
korre.pororo_ner(sent2)

[('모토로라 레이저 M', 'ARTIFACT'),
 ('는', 'O'),
 (' ', 'O'),
 ('모토로라 모빌리티', 'ORGANIZATION'),
 ('에서', 'O'),
 (' ', 'O'),
 ('제조/판매하는', 'O'),
 (' ', 'O'),
 ('안드로이드', 'TERM'),
 (' ', 'O'),
 ('스마트폰', 'TERM'),
 ('이다.', 'O')]

In [211]:
korre.pororo_ner(sent3)

[('징크스는', 'O'),
 (' ', 'O'),
 ('리그오브레전드', 'ARTIFACT'),
 (' ', 'O'),
 ('캐릭터이다.', 'O')]

In [212]:
korre.ner(sent1)

[('아이폰', 'ARTIFACT', [0, 3]),
 ('애플', 'ORGANIZATION', [5, 7]),
 ('스마트폰', 'TERM', [15, 19])]

In [214]:
korre.ner(sent2)

[('모토로라 레이저 M', 'ARTIFACT', [0, 10]),
 ('모토로라 모빌리티', 'ORGANIZATION', [12, 21]),
 ('안드로이드', 'TERM', [32, 37]),
 ('스마트폰', 'TERM', [38, 42])]

In [215]:
korre.ner(sent3)

[('리그오브레전드', 'ARTIFACT', [5, 12])]

In [216]:
korre.get_all_entity_pairs(sent1)

[(('아이폰', 'ARTIFACT', [0, 3]), ('애플', 'ORGANIZATION', [5, 7])),
 (('아이폰', 'ARTIFACT', [0, 3]), ('스마트폰', 'TERM', [15, 19])),
 (('애플', 'ORGANIZATION', [5, 7]), ('아이폰', 'ARTIFACT', [0, 3])),
 (('애플', 'ORGANIZATION', [5, 7]), ('스마트폰', 'TERM', [15, 19])),
 (('스마트폰', 'TERM', [15, 19]), ('아이폰', 'ARTIFACT', [0, 3])),
 (('스마트폰', 'TERM', [15, 19]), ('애플', 'ORGANIZATION', [5, 7]))]

In [219]:
for item in korre.get_all_entity_pairs(sent2):
    print(item)

(('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21]))
(('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('안드로이드', 'TERM', [32, 37]))
(('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('스마트폰', 'TERM', [38, 42]))
(('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10]))
(('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('안드로이드', 'TERM', [32, 37]))
(('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('스마트폰', 'TERM', [38, 42]))
(('안드로이드', 'TERM', [32, 37]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10]))
(('안드로이드', 'TERM', [32, 37]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21]))
(('안드로이드', 'TERM', [32, 37]), ('스마트폰', 'TERM', [38, 42]))
(('스마트폰', 'TERM', [38, 42]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10]))
(('스마트폰', 'TERM', [38, 42]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21]))
(('스마트폰', 'TERM', [38, 42]), ('안드로이드', 'TERM', [32, 37]))


In [221]:
for item in korre.get_all_entity_pairs(sent3):
    print(item)

In [222]:
korre.get_all_inputs(sent1)

[['아이폰은 애플에서 만들어진 스마트폰이다.', [0, 3], [5, 7]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [0, 3], [15, 19]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [5, 7], [0, 3]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [5, 7], [15, 19]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [15, 19], [0, 3]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [15, 19], [5, 7]]]

In [223]:
korre.get_all_inputs(sent2)

[['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [32, 37]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [32, 37]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [32, 37]]]

In [224]:
korre.get_all_inputs(sent3)

[]

In [229]:
for i in range(len(korre.get_all_inputs(sent1))):
    print(korre.entity_markers_added(*korre.get_all_inputs(sent1)[i]))

[E1] 아이폰 [/E1] 은  [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.
[E1] 아이폰 [/E1] 은 애플에서 만들어진  [E2] 스마트폰 [/E2] 이다.
[E2] 아이폰 [/E2] 은  [E1] 애플 [/E1] 에서 만들어진 스마트폰이다.
아이폰은  [E1] 애플 [/E1] 에서 만들어진  [E2] 스마트폰 [/E2] 이다.
[E2] 아이폰 [/E2] 은 애플에서 만들어진  [E1] 스마트폰 [/E1] 이다.
아이폰은  [E2] 애플 [/E2] 에서 만들어진  [E1] 스마트폰 [/E1] 이다.


In [232]:
for i in korre.get_all_inputs(sent1):
    print(korre.infer(*i))

[('아이폰', '애플', '제조사')]
[('아이폰', '스마트폰', '다음의 하위 개념임')]
[('애플', '아이폰', '제품')]
[('애플', '스마트폰', '제품')]
[]
[('스마트폰', '애플', '제조사')]


In [21]:
for i in range(len(korre.get_all_inputs(sent1))):
    print(korre.infer(korre.entity_markers_added(*korre.get_all_inputs(sent1)[i]), entity_markers_included=True))

TypeError: string indices must be integers

In [25]:
korre.infer('[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', entity_markers_included=True)

[('아이폰', '애플', '해당 개체의 제조사(manufacturer)')]

In [26]:
sent = '[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.'
korre.get_all_inputs(sent)

[['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [5, 8], [22, 24]],
 ['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [5, 8], [39, 43]],
 ['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [22, 24], [5, 8]],
 ['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [22, 24], [39, 43]],
 ['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [39, 43], [5, 8]],
 ['[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', [39, 43], [22, 24]]]

In [28]:
korre.infer(sent, entity_markers_included=True)

[('아이폰', '애플', '해당 개체의 제조사(manufacturer)')]

In [29]:
korre.infer('[E1] 아이폰[/E1] 은  [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.', entity_markers_included=True)

[('아이폰', '애플', '해당 개체의 제조사(manufacturer)')]

In [31]:
korre.infer('[E1] 징크스[/E1]는 [E2]리그오브레전드 [/E2] 캐릭터이다.', entity_markers_included=True)

[('징크스', '리그오브레전드', '해당 개체가 다음 작품에 등장함(present in work)')]

In [32]:
korre.infer('사미라는 리그오브레전드 캐릭터로, 주로 총과 칼을 이용하여 전투를 하는 캐릭터이다.')

[('사미라', '리그오브레전드', '해당 개체가 다음 작품에 등장함(present in work)'),
 ('리그오브레전드', '사미라', '해당 개체가 다음으로 이루어져 있음(has part)'),
 ('리그오브레전드', '총', '해당 개체가 다음으로 이루어져 있음(has part)'),
 ('리그오브레전드', '칼', '해당 개체가 다음으로 이루어져 있음(has part)'),
 ('총', '리그오브레전드', '다음과 다르지만 같은 의미인 것처럼 혼동되는 항목(different from)'),
 ('총', '칼', '다음과 다르지만 같은 의미인 것처럼 혼동되는 항목(different from)'),
 ('칼', '총', '다음과 다르지만 같은 의미인 것처럼 혼동되는 항목(different from)')]

In [33]:
import requests

r = requests.get('https://huggingface.co/datawhales/korean-relation-extraction/resolve/main/pytorch_model.bin')

In [37]:
torch.hub.load_state_dict_from_url('https://huggingface.co/datawhales/korean-relation-extraction/resolve/main/pytorch_model.bin')

Downloading: "https://huggingface.co/datawhales/korean-relation-extraction/resolve/main/pytorch_model.bin" to /Users/datawhales/.cache/torch/hub/checkpoints/pytorch_model.bin


  0%|          | 0.00/388M [00:00<?, ?B/s]

RuntimeError: Only one file(not dir) is allowed in the zipfile

In [43]:
!pip install torch==1.7.0

     |████████████████████████████████| 108.1 MB 29.7 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.7.0 requires torch==1.6.0, but you have torch 1.7.0 which is incompatible.
pororo 0.4.2 requires torch==1.6.0, but you have torch 1.7.0 which is incompatible.


In [45]:
import torch

torch.__version__

'1.6.0'

In [192]:
korre.infer('[E1] 아이폰 [/E1] 은 [E2] 애플 [/E2] 에서 만들어진 [E2] 스마트폰 [/E2] 이다.', entity_markers_included=True)

Exception: Incorrect number of entity marker tokens('[E1]', '[/E1]', '[E2]', '[/E2]').

In [122]:
sent = '아이폰은 애플에서 만들어진 스마트폰이다.'

In [124]:
korre.pororo_ner(sent)

[('아이폰', 'ARTIFACT'),
 ('은', 'O'),
 (' ', 'O'),
 ('애플', 'ORGANIZATION'),
 ('에서', 'O'),
 (' ', 'O'),
 ('만들어진', 'O'),
 (' ', 'O'),
 ('스마트폰', 'TERM'),
 ('이다.', 'O')]

In [125]:
korre.ner(sent)

[('아이폰', 'ARTIFACT', [0, 3]),
 ('애플', 'ORGANIZATION', [5, 7]),
 ('스마트폰', 'TERM', [15, 19])]

In [126]:
korre.get_all_entity_pairs(sent)

[(('아이폰', 'ARTIFACT', [0, 3]), ('애플', 'ORGANIZATION', [5, 7])),
 (('아이폰', 'ARTIFACT', [0, 3]), ('스마트폰', 'TERM', [15, 19])),
 (('애플', 'ORGANIZATION', [5, 7]), ('아이폰', 'ARTIFACT', [0, 3])),
 (('애플', 'ORGANIZATION', [5, 7]), ('스마트폰', 'TERM', [15, 19])),
 (('스마트폰', 'TERM', [15, 19]), ('아이폰', 'ARTIFACT', [0, 3])),
 (('스마트폰', 'TERM', [15, 19]), ('애플', 'ORGANIZATION', [5, 7]))]

In [130]:
korre.get_all_inputs(sent)

[['아이폰은 애플에서 만들어진 스마트폰이다.', [0, 3], [5, 7]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [0, 3], [15, 19]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [5, 7], [0, 3]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [5, 7], [15, 19]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [15, 19], [0, 3]],
 ['아이폰은 애플에서 만들어진 스마트폰이다.', [15, 19], [5, 7]]]

In [131]:
korre.get_all_inputs(sent)[0]

['아이폰은 애플에서 만들어진 스마트폰이다.', [0, 3], [5, 7]]

In [134]:
korre.entity_markers_added(*korre.get_all_inputs(sent)[0])

'[E1] 아이폰 [/E1] 은  [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.'

In [138]:
3 in torch.tensor([[1,2]])

False

In [146]:
new_sent = '[E1] 아이폰 [/E1] 은  [E2] 애플 [/E2] 에서 만들어진 스마트폰이다.'

new_sent.split()

['[E1]', '아이폰', '[/E1]', '은', '[E2]', '애플', '[/E2]', '에서', '만들어진', '스마트폰이다.']

In [160]:
new_sent = '[E1]아이폰[/E1]은 [E2]애플 [/E2] 에서 만들어진 스마트폰이다.'
korre.tokenizer.decode(korre.tokenizer(new_sent)['input_ids'], skip_special_tokens=True)

'아이폰 은 애플 에서 만들어진 스마트폰이다.'

In [182]:
korre.tokenizer(new_sent)['input_ids'].count(20000)

1

In [ ]:
subj_id, obj_id = [], []


    

In [169]:
subj_start_id = korre.tokenizer(new_sent)['input_ids'].index(20000)
subj_end_id = korre.tokenizer(new_sent)['input_ids'].index(20001)
obj_start_id = korre.tokenizer(new_sent)['input_ids'].index(20002)
obj_end_id = korre.tokenizer(new_sent)['input_ids'].index(20003)

In [170]:
subj_name = korre.tokenizer.decode(korre.tokenizer(new_sent)['input_ids'][subj_start_id+1:subj_end_id])
obj_name = korre.tokenizer.decode(korre.tokenizer(new_sent)['input_ids'][obj_start_id+1:obj_end_id])

In [171]:
subj_name

'아이폰'

In [172]:
obj_name

'애플'

In [174]:
korre.tokenizer.decode([14071])

'아이폰'

In [ ]:
tokenizer.decode()

In [9]:
korre.infer('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [12, 21])

../kornre/pretrain.py:146: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:766.)
  h_start_pos_tensor = (input_ids == 20000).nonzero()


[('모토로라 레이저 M', '모토로라 모빌리티', '제조사')]

In [10]:
korre.infer('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [32,42])

[('모토로라 레이저 M', '안드로이드 스마트폰', '다음의 하위 개념임')]

In [34]:
korre.infer('헥토르는 동생 데이포보스가 자신을 도와주러 온 것으로 믿고 아킬레우스와 맞서 싸웠다.', [0, 3], [8,11])

[('헥토르', '데이포', '친형제자매')]

In [12]:
korre.infer('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.')

[('모토로라 레이저 M', '모토로라 모빌리티', '제조사'),
 ('모토로라 레이저 M', '안드로이드', '다음의 하위 개념임'),
 ('모토로라 레이저 M', '스마트폰', '다음의 하위 개념임'),
 ('모토로라 모빌리티', '안드로이드', '제품'),
 ('모토로라 모빌리티', '스마트폰', '제품'),
 ('안드로이드', '스마트폰', '다음의 하위 개념임'),
 ('스마트폰', '모토로라 레이저 M', '다음으로 이루어져 있음'),
 ('스마트폰', '안드로이드', '다음과는 확실히 다름')]

In [58]:
korre.infer('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0,10],[38,42])

[('모토로라 레이저 M', '스마트폰', '다음의 하위 개념임')]

In [14]:
korre.get_all_inputs('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.')

[['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [32, 37]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [0, 10], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [32, 37]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [12, 21], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [32, 37], [38, 42]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [0, 10]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [12, 21]],
 ['모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.', [38, 42], [32, 37]]]

In [16]:
korre.get_all_entity_pairs('모토로라 레이저 M는 모토로라 모빌리티에서 제조/판매하는 안드로이드 스마트폰이다.')

[(('모토로라 레이저 M', 'ARTIFACT', [0, 10]),
  ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
 (('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('안드로이드', 'TERM', [32, 37])),
 (('모토로라 레이저 M', 'ARTIFACT', [0, 10]), ('스마트폰', 'TERM', [38, 42])),
 (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]),
  ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
 (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('안드로이드', 'TERM', [32, 37])),
 (('모토로라 모빌리티', 'ORGANIZATION', [12, 21]), ('스마트폰', 'TERM', [38, 42])),
 (('안드로이드', 'TERM', [32, 37]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
 (('안드로이드', 'TERM', [32, 37]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
 (('안드로이드', 'TERM', [32, 37]), ('스마트폰', 'TERM', [38, 42])),
 (('스마트폰', 'TERM', [38, 42]), ('모토로라 레이저 M', 'ARTIFACT', [0, 10])),
 (('스마트폰', 'TERM', [38, 42]), ('모토로라 모빌리티', 'ORGANIZATION', [12, 21])),
 (('스마트폰', 'TERM', [38, 42]), ('안드로이드', 'TERM', [32, 37]))]

In [27]:
korre.infer('아이폰은 애플에서 만들어진 스마트폰이다.')

[('아이폰', '애플', '제조사'),
 ('아이폰', '스마트폰', '다음의 하위 개념임'),
 ('애플', '아이폰', '제품'),
 ('애플', '스마트폰', '제품'),
 ('스마트폰', '애플', '제조사')]

In [72]:
tmp_sent = '징크스는 리그오브레전드에 등장하는 캐릭터이다.'
korre.ner(tmp_sent)

[('징크스는', 'O', 4),
 (' ', 'O', 1),
 ('리그오브레전드', 'ARTIFACT', 7),
 ('에', 'O', 1),
 (' ', 'O', 1),
 ('등장하는', 'O', 4),
 (' ', 'O', 1),
 ('캐릭터이다.', 'O', 6)]

In [80]:
korre.infer('징크스는 리그오브레전드에 등장하는 캐릭터이다.', [0,6],[0,2])

[]

In [ ]:
korre.infer()

In [74]:
korre.infer('징크스는 리그오브레전드 게임에 등장하는 캐릭터이다.', [0, 3], [14, 16])

[]

In [65]:
'징크스는 리그오브레전드에 등장하는 캐릭터이다.'[5:13]

'리그오브레전드에'

In [60]:
korre.infer('공부하는 곳인 명륜당이 앞에, 사당인 대성전 뒤에 있는 전학후묘의 형태로 향교의 일반적인 배치를 따르고 있다.')

[('명륜당', '앞', '반대 개념'),
 ('명륜당', '뒤', '명칭의 유래'),
 ('앞', '뒤', '반대 개념'),
 ('대성전', '뒤', '명칭의 유래'),
 ('뒤', '앞', '반대 개념')]